In [19]:
using CSV, DataFrames, JuMP, Gurobi, Formatting, NPZ

In [20]:
import Pkg;
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("Formatting")
Pkg.add("NPZ")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


# Loading Data

In [21]:
time_s2d_df = CSV.read("csv/time_srcs_to_dests.csv", DataFrame, header=false);
length_s2d_df = CSV.read("csv/length_srcs_to_dests.csv", DataFrame, header=false);

time_d2d_df = CSV.read("csv/time_dests_to_dests.csv", DataFrame, header=false);
length_d2d_df = CSV.read("csv/length_dests_to_dests.csv", DataFrame, header=false);

In [22]:
first(time_d2d_df, 2)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,50.1583,33.3067,25.5429,27.6542,22.7042,28.0369,22.2431,22.1509
2,50.1583,0.0,20.9763,43.2164,41.9561,37.0061,26.3583,40.6364,40.262


In [23]:
first(length_d2d_df, 2)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,5.36814,3.37471,2.9547,2.86724,2.34539,2.84076,2.55595,2.59004
2,5.36814,0.0,2.32505,4.58539,4.39593,3.89126,2.74863,4.30058,4.2832


In [24]:
n_src = nrow(time_s2d_df);
n_dst = ncol(time_s2d_df);

all_src = 1:n_src;
all_dst = 1:n_dst;

println("all_src: $all_src, all_dst: $all_dst")

all_src: 1:3, all_dst: 1:9


In [25]:
T_s2d = Matrix(time_s2d_df);
L_s2d = Matrix(length_s2d_df);
T_d2d = Matrix(time_d2d_df);
L_d2d = Matrix(length_d2d_df);

println("T_s2d: $(size(T_s2d)), L_s2d: $(size(L_s2d)), T_d2d: $(size(T_d2d)), L_d2d: $(size(L_d2d))")

T_s2d: (3, 9), L_s2d: (3, 9), T_d2d: (9, 9), L_d2d: (9, 9)


# Problem
- We now additionally consider that each robot has a fixed starting source.
- The number of robots is less than the number of destinations.
- Instead of minimizing the total time / distance, we want to minimize the maximum time / distance for any single robot.

In [26]:
S = [2, 1, 1];
@assert size(S) == (n_src,)
@assert sum(S) < n_dst

n_robots = sum(S)
all_robots = 1:n_robots;

println("There are a $n_dst destinations but only $n_robots robots to visit them all.")

There are a 9 destinations but only 4 robots to visit them all.


In [27]:
# We assume that each robot visits a maximum of n_steps destinations.
n_steps = 3;
all_steps = 1:n_steps;
later_steps = 1:(n_steps-1);

println("n_steps: $n_steps, all_steps: $all_steps, later_steps: $later_steps")

n_steps: 3, all_steps: 1:3, later_steps: 1:2


# Minimum Time

In [28]:
model1 = Model(Gurobi.Optimizer)

@variable(model1, x0[all_robots, all_src, all_dst] >= 0, Bin);
@variable(model1, x1[all_robots, later_steps, all_dst, all_dst] >= 0, Bin);
@variable(model1, maxtime >= 0);

time_fromsrc = [sum(sum(T_s2d[ii, jj] * x0[rr, ii, jj] for ii in all_src) for jj in all_dst) for rr in all_robots];
time_fromdst = [sum(sum(sum(T_d2d[ii, jj] * x1[rr, kk, ii, jj] for ii in all_dst) for jj in all_dst) for kk in later_steps) for rr in all_robots];
time_total = [time_fromsrc[rr] + time_fromdst[rr] for rr in all_robots];

@objective(model1, Min, maxtime);

# The max time is larger than the times for each robot.
@constraint(model1, maxtimelb[rr in all_robots], maxtime >= time_total[rr]);

# Each destination must be served by at least one source.
@constraint(model1, demand[jj in all_dst], sum(sum(x0[rr, ii, jj] for ii in all_src) for rr in all_robots) + sum(sum(sum(x1[rr, kk, ii, jj] for ii in all_dst) for kk in later_steps) for rr in all_robots) >= 1);

# Continuity constraints. x0_{r, ij} = x1_{r, jl}
@constraint(model1, cont0[rr in all_robots, jj in all_dst], sum(x0[rr, ii, jj] for ii in all_src) == sum(x1[rr, 1, jj, ll] for ll in all_dst));
# Continuity constraints. x1_{r, ij} = x2_{r, jl}
@constraint(model1, cont1[rr in all_robots, kk in 1:(n_steps-2), jj in all_dst], sum(x1[rr, kk, ii, jj] for ii in all_dst) == sum(x1[rr, kk + 1, jj, ll] for ll in all_dst));

# Each robot must start from its assigned source.
@constraint(model1, start[ii in all_src], sum(sum(x0[rr, ii, jj] for jj in all_dst) for rr in all_robots) == S[ii]);

# Solve.
optimize!(model1);
solution_summary(model1)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-07
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 88 rows, 757 columns and 2632 nonzeros
Model fingerprint: 0x976e6d5d
Variable types: 1 continuous, 756 integer (756 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 106.3848441
Presolve time: 0.01s
Presolved: 88 rows, 757 columns, 2632 nonzeros
Variable types: 1 continuous, 756 integer (756 binary)

Root relaxation: objective 1.382540e+01, 127 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

* Solver : Gurobi

* Status
  Result count       : 10
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 3.23048e+01
  Objective bound    : 3.23048e+01
  Relative gap       : 0.00000e+00
  Dual objective value : 3.23048e+01

* Work counters
  Solve time (sec)   : 3.15875e-01
  Barrier iterations : 0
  Node count         : 256


In [29]:
# Save solution.
x1_0 = Array(value.(x0));
x1_1 = Array(value.(x1));

npzwrite("sols/problem4_time.npz", Dict("x0" => x1_0, "x1" => x1_1, "objective" => objective_value(model1)))

# Minimum Distance

In [30]:
model2 = Model(Gurobi.Optimizer)

@variable(model2, x0[all_robots, all_src, all_dst] >= 0, Bin);
@variable(model2, x1[all_robots, later_steps, all_dst, all_dst] >= 0, Bin);
@variable(model2, maxdist >= 0);

dist_fromsrc = [sum(sum(L_s2d[ii, jj] * x0[rr, ii, jj] for ii in all_src) for jj in all_dst) for rr in all_robots];
dist_fromdst = [sum(sum(sum(L_d2d[ii, jj] * x1[rr, kk, ii, jj] for ii in all_dst) for jj in all_dst) for kk in later_steps) for rr in all_robots];
dist_total = [dist_fromsrc[rr] + dist_fromdst[rr] for rr in all_robots];

@objective(model2, Min, maxdist);

# The max dist is larger than the dists for each robot.
@constraint(model2, maxdistlb[rr in all_robots], maxdist >= dist_total[rr]);

# Each destination must be served by at least one source.
@constraint(model2, demand[jj in all_dst], sum(sum(x0[rr, ii, jj] for ii in all_src) for rr in all_robots) + sum(sum(sum(x1[rr, kk, ii, jj] for ii in all_dst) for kk in later_steps) for rr in all_robots) >= 1);

# Continuity constraints. x0_{r, ij} = x1_{r, jl}
@constraint(model2, cont0[rr in all_robots, jj in all_dst], sum(x0[rr, ii, jj] for ii in all_src) == sum(x1[rr, 1, jj, ll] for ll in all_dst));
# Continuity constraints. x1_{r, ij} = x2_{r, jl}
@constraint(model2, cont1[rr in all_robots, kk in 1:(n_steps-2), jj in all_dst], sum(x1[rr, kk, ii, jj] for ii in all_dst) == sum(x1[rr, kk + 1, jj, ll] for ll in all_dst));

# Each robot must start from its assigned source.
@constraint(model2, start[ii in all_src], sum(sum(x0[rr, ii, jj] for jj in all_dst) for rr in all_robots) == S[ii]);

# Solve.
optimize!(model2);
solution_summary(model2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-07
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 88 rows, 757 columns and 2632 nonzeros
Model fingerprint: 0xb1e804aa
Variable types: 1 continuous, 756 integer (756 binary)
Coefficient statistics:
  Matrix range     [3e-01, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 11.2678852
Presolve time: 0.01s
Presolved: 88 rows, 757 columns, 2632 nonzeros
Variable types: 1 continuous, 756 integer (756 binary)

Root relaxation: objective 1.466691e+00, 203 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent  

* Solver : Gurobi

* Status
  Result count       : 10
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 3.42408e+00
  Objective bound    : 3.42408e+00
  Relative gap       : 0.00000e+00
  Dual objective value : 3.42408e+00

* Work counters
  Solve time (sec)   : 2.99309e-01
  Barrier iterations : 0
  Node count         : 197


In [31]:
# Save solution.
x2_0 = Array(value.(x0));
x2_1 = Array(value.(x1));

npzwrite("sols/problem4_dist.npz", Dict("x0" => x2_0, "x1" => x2_1, "objective" => objective_value(model2)))

# Compare the solutions.

In [32]:
display(sum(sum(sum(x1_0 - x2_0))))
display(sum(sum(sum(x1_1 - x2_1))))

0.0

0.0